In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [16]:
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

In [20]:
algo = SVD()
algo.fit(trainset)

In [21]:
predictions = algo.test(testset)
print('prediction type :', type(predictions), 'size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'> size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.6187935076171995, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.841440897358398, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.150517687818901, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.6417411256334526, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.7187537328890485, details={'was_impossible': False})]

In [22]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.6187935076171995),
 ('882', '291', 3.841440897358398),
 ('535', '507', 4.150517687818901)]

In [23]:
uid=  str(195)
iid = str(302)
pred = algo.predict(uid, iid)

print(pred)

user: 195        item: 302        r_ui = None   est = 4.07   {'was_impossible': False}


In [24]:
accuracy.rmse(predictions)

RMSE: 0.9514


0.9514245534839372

In [25]:
import pandas as pd
ratings = pd.read_csv('./ratings.csv')
ratings.to_csv('./ratings_noh.csv', index=False, header=False)

In [26]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep = ',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('./ratings_noh.csv', reader=reader)

In [27]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [28]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ratings.csv')
reader=  Reader(rating_scale=(0.5, 5.0))

data  = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']],reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [29]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('./ratings.csv')
rader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId','movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8684  0.8756  0.8691  0.8763  0.8778  0.8734  0.0039  
MAE (testset)     0.6673  0.6739  0.6689  0.6741  0.6708  0.6710  0.0027  
Fit time          5.41    5.23    5.58    5.46    5.24    5.39    0.14    
Test time         0.14    0.30    0.15    0.16    0.31    0.21    0.08    


{'test_rmse': array([0.86837936, 0.87564825, 0.86907843, 0.87631585, 0.87780565]),
 'test_mae': array([0.66734528, 0.67390695, 0.66893115, 0.67408371, 0.67077005]),
 'fit_time': (5.409404039382935,
  5.23103141784668,
  5.583215236663818,
  5.464938640594482,
  5.239124059677124),
 'test_time': (0.1386270523071289,
  0.3041861057281494,
  0.14657020568847656,
  0.15658092498779297,
  0.30518388748168945)}

In [30]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs':[20,40,60], 'n_factors':[50,100,200]}

gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8760798739356378
{'n_epochs': 20, 'n_factors': 50}


In [32]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data_folds = DatasetAutoFolds(ratings_file='./ratings_noh.csv', reader=reader)

trainset = data_folds.build_full_trainset()

In [33]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [35]:
movies =pd.read_csv('./movies.csv')

movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() == 0 :
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')
    
print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [36]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [41]:
def get_unseen_surprise(ratings, movies, userId) :
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    
    total_movies = movies['movieId'].tolist()
    
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies),
         '전체 영화 수:', len(total_movies))
    
    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [43]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10) :
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    
    def sortkey_est(pred) :
        return pred.est
    
    predictions.sort(key=sortkey_est, reverse=True)
    
    top_predictions = predictions[:top_n]
    
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating  = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [(id, title, rating) for id, title, rating in
                      zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds : 
    print(top_movie[1], ":", top_movie[2])
    

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
